Enoncé du TD (Analyse de sentiment, le retour)
=========================
Le TD consiste à redéfinir le projet 1 : prédicteur naïf de la polarité d'une critique de film tirée de IMDB 
anglais. Le modèle est un modèle de régression logistique traditionnel implémenté avec pytorch mais cette fois-ci le texte est représenté à l'aide d'un LSTM.

Le jeu de données est à télécharger depuis le site suivant : http://ai.stanford.edu/~amaas/data/sentiment/

Le TD consiste à remplacer le modèle utilisé dans le TD 1 (où $\Phi(\mathbf{x})$ représente un vecteur de fréquences des mots dans un texte):
\begin{align}
P(Y=1 | \Phi(\mathbf{x}) ;  \boldsymbol\theta) &= \frac{exp(\mathbf{w}^T\Phi(\mathbf{x}))}{1+exp(\mathbf{w}^T\Phi(\mathbf{x}))}
\end{align}
par le modèle suivant :
\begin{align}
P(Y=1 | \mathbf{x} ;  \boldsymbol\theta) &= \frac{exp(\mathbf{w}^T \mathbf{h})}{1+exp(\mathbf{w}^T\mathbf{h})}\\
\mathbf{h} &= \text{LSTM}(\mathbf{e})\\
\mathbf{e} &= \text{embedding}(\mathbf{x})
\end{align}

où $\mathbf{e}$ est un vecteur d'embeddings (un embedding par mot du texte à traiter) mais $\mathbf{h}$ est un vecteur de réels qui représente la mémoire du LSTM lors de la dernière prédiction.

Plan détaillé :
 * Observez que la fonction de lecture de données ne renvoie plus un dictionnaire mais une liste des mots présents dans le texte
 * Remplissez la fonction `code_sequence` pour qu'elle renvoie une liste d'entiers qui correspond à la séquence des codes de mots du texte. La fonction renvoie cette liste sous forme de `torch.tensor`.
 * Mettez à jour les méthodes du modèle pour coder le texte à l'aide d'un LSTM. Il s'agit des méthodes : `allocate_structure`, `forward` et éventuellement `train`. 
 * Vous pouvez également remplacer le LSTM par un Bi-LSTM, mais c'est un peu plus difficile à programmer (voir sur internet).
 * Vous verrez que l'entrainement est très long si réalisé naivement. Vous pouvez mettre au point une méthode qui (a) filtre le vocabulaire pour réduire la taille des textes et qui (b) organise l'entrainement en "mini-batchs" (voir sur internet, **exercice plus difficile**)
 
Le rendu attendu est une copie de ce notebook qui aura été complétée. Profitez du framework notebook pour commenter vos réponses si besoin.







In [1]:
import os
import os.path
import random

random.seed(1)


#loads data from disk
def load_dataset(dir_path,ref_label):
    """
    Loads a dataset from a directory path and 
    returns a list of couples (list of words,ref_label) one for each text
    """
    dpath    = os.path.abspath(dir_path)
    data_set = [] 
    for f in os.listdir(dpath):
        filepath    = os.path.join(dpath,f)
        file_stream = open(filepath)
        text        = file_stream.read().split()
        data_set.append((text,ref_label))
        file_stream.close()
    return data_set
    
trainpos = load_dataset("/Users/bcrabbe/parsing-at-diderot/data/aclImdb/train/pos",1)
trainneg = load_dataset("/Users/bcrabbe/parsing-at-diderot/data/aclImdb/train/neg",0)
trainset = trainpos
trainset.extend(trainneg)
random.shuffle(trainset)

In [2]:
testpos = load_dataset("/Users/bcrabbe/parsing-at-diderot/data/aclImdb/test/pos",1)
testneg = load_dataset("/Users/bcrabbe/parsing-at-diderot/data/aclImdb/test/neg",0)
testset = testpos
testset.extend(testneg)
random.shuffle(testset)

trainset = trainset
validset = testset[:5000]
testset  = testset[5000:]


In [3]:
from collections import Counter

#creates a dict that maps each known word string to a unique integer
def make_w2idx(dataset,vocab_size=None,unk_symbol=None):
    wordcounts = Counter()
    for X,Y in dataset:
        wordcounts.update([word for word in X])
        
    print('Capping a vocab of size %d to size %d'%(len(wordcounts),vocab_size))
    wordset =[ word for word,c in wordcounts.most_common(vocab_size) ]
    
    if unk_symbol:
        wordset.append(unk_symbol)
        
    return dict(zip(wordset,range(len(wordset))))   

def code_sequence(symlist,w2idx,unk_symbol=None,maxlen=100):       
    def normal_form(symbol):
        return symbol if symbol in w2idx else unk_symbol
    
    if maxlen > 0:
        symlist = symlist[:maxlen]
    
    code_list = [ w2idx[normal_form(symbol)] for symbol in symlist ]
    return torch.tensor(code_list, dtype=torch.long)

def vectorize_target(ylabel):
     return torch.tensor(float(ylabel))
  

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

class SentimentAnalyzer(nn.Module): 
        def __init__(self, embedding_dim,lstm_memory_dim):
            
            super(SentimentAnalyzer, self).__init__()
            self.embedding_dim        = embedding_dim
            self.lstm_memory_dim      = lstm_memory_dim
            self.allocate_structure(1,1)
           
        
        def allocate_structure(self,vocab_size, num_labels):
            
            self.E      = nn.Embedding(vocab_size,self.embedding_dim)
            self.bilstm = nn.LSTM(self.embedding_dim, self.lstm_memory_dim,num_layers=1,bidirectional=True) 
            self.W      = nn.Linear(self.lstm_memory_dim*2, num_labels)    
             
            
        def forward(self, xinput):
            xembedded                    = self.E(xinput)
            lstm_out, (hiddwen,cell)      = self.bilstm(xembedded.view(len(xinput), 1, -1), None)
            return F.sigmoid(self.W(hidden.view(1,self.lstm_memory_dim*2))) #sigmoid is the logistic activation
        
        def train(self,train_set,dev_set,learning_rate,epochs,vocab_size=1000,unk_symbol='<unk>'):
            
            self.w2idx = make_w2idx(train_set,vocab_size,unk_symbol)
            self.unk   = unk_symbol
            self.allocate_structure(len(self.w2idx),1)
            loss_func  = nn.BCELoss() #remind that minimizing Binary Cross Entropy <=> minimizing NLL
            optimizer  = optim.Adam(self.parameters(), lr=learning_rate)
            
            min_loss = np.iinfo(np.int32).max
            for epoch in range(epochs):
                global_logloss = 0.0
                for X, Y in train_set: 
                    self.zero_grad()
                    xvec            = code_sequence(X,self.w2idx,unk_symbol)
                    yvec            = vectorize_target(Y)
                    prob            = self.forward(xvec).squeeze()
                    loss            = loss_func(prob,yvec)
                    loss.backward()
                    optimizer.step()
                    global_logloss += loss.item()
                print("Epoch %d, mean cross entropy (train) = %f"%(epoch,global_logloss/len(train_set)))

                with torch.no_grad():
                    global_logloss = 0.0
                    for X, Y in dev_set: 
                        xvec            = code_sequence(X,self.w2idx,unk_symbol)
                        yvec            = vectorize_target(Y)
                        prob            = self(xvec).squeeze()
                        loss            = loss_func(prob,yvec)
                        global_logloss += loss.item()
                    print("Epoch %d, mean cross entropy (dev)   = %f\n"%(epoch,global_logloss/len(dev_set)))
                    if global_logloss < min_loss:
                        torch.save(self.state_dict(), 'sentiment_model.wt')
            self.load_state_dict(torch.load('sentiment_model.wt'))
            
        def run_test(self,test_set):
            
            ncorrect = 0
            with torch.no_grad():
                for X, Y in test_set: 
                    xvec            = code_sequence(X,self.w2idx,self.unk)
                    prob            = self(xvec).squeeze()
                    if Y == 1 and prob > 0.5:
                        ncorrect += 1
                    if Y == 0 and prob <= 0.5: 
                        ncorrect += 1
            print("Test Accurracy",ncorrect/len(test_set))
            

In [5]:
sent = SentimentAnalyzer(32,64)
sent.train(trainset,validset,0.001,3,vocab_size=5000)

Capping a vocab of size 280617 to size 5000
Epoch 0, mean cross entropy (train) = 0.610629
Epoch 0, mean cross entropy (dev)   = 0.483787

Epoch 1, mean cross entropy (train) = 0.427120
Epoch 1, mean cross entropy (dev)   = 0.434813

Epoch 2, mean cross entropy (train) = 0.342622
Epoch 2, mean cross entropy (dev)   = 0.470207



In [6]:
sent.run_test(testset)

Test Accurracy 0.7865
